In [8]:
# ------------------ 📦 IMPORTS ------------------
import os
import json
import re
import datetime
import pytz
import pickle
import speech_recognition as sr
import pyttsx3
import google.generativeai as genai
from dotenv import load_dotenv
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from langdetect import detect

# ------------------ 🔐 LOAD ENV VARIABLES ------------------
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SCOPES = ['https://www.googleapis.com/auth/calendar']
TIMEZONE = "Asia/Kolkata"

# ------------------ 🔊 TEXT TO SPEECH ------------------
engine = pyttsx3.init()
def speak(text, lang="en"):
    print("🤖 Bot:", text)
    engine.say(text)
    engine.runAndWait()

# ------------------ 🎙️ SPEECH TO TEXT ------------------
def listen():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 Adjusting for ambient noise...")
        recognizer.adjust_for_ambient_noise(source)
        print("🟢 Listening...")
        try:
            audio = recognizer.listen(source, timeout=10)
            print("⏳ Transcribing...")
            text = recognizer.recognize_google(audio, language='en-IN')
            lang = detect(text)
            print("🗣️ You said:", text)
            return text, lang
        except Exception as e:
            print("❌ STT Error:", e)
            return "", "en"

# ------------------ 💾 CONTEXT MEMORY ------------------
def save_context(ctx):
    with open("context.pkl", "wb") as f:
        pickle.dump(ctx, f)

def load_context():
    try:
        with open("context.pkl", "rb") as f:
            return pickle.load(f)
    except:
        return None

# ------------------ 📅 GOOGLE CALENDAR AUTH ------------------
def get_calendar_service():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('calendar', 'v3', credentials=creds)

# ------------------ 📅 GET FREE & BUSY SLOTS ------------------
def get_free_and_busy_slots(service, date_str, duration_min=60):
    tz = pytz.timezone(TIMEZONE)
    start_of_day = tz.localize(datetime.datetime.strptime(f"{date_str} 00:00", "%Y-%m-%d %H:%M"))
    end_of_day = start_of_day + datetime.timedelta(days=1)

    result = service.freebusy().query(
        body={
            "timeMin": start_of_day.isoformat(),
            "timeMax": end_of_day.isoformat(),
            "timeZone": TIMEZONE,
            "items": [{"id": "primary"}]
        }
    ).execute()

    busy_times = result['calendars']['primary'].get('busy', [])
    free_slots = []
    current = start_of_day

    for busy in busy_times:
        start = datetime.datetime.fromisoformat(busy['start']).astimezone(tz)
        if (start - current).total_seconds() >= duration_min * 60:
            free_slots.append((current.strftime('%H:%M'), start.strftime('%H:%M')))
        current = datetime.datetime.fromisoformat(busy['end']).astimezone(tz)

    if (end_of_day - current).total_seconds() >= duration_min * 60:
        free_slots.append((current.strftime('%H:%M'), end_of_day.strftime('%H:%M')))

    busy_slots = [(datetime.datetime.fromisoformat(b['start']).astimezone(tz).strftime('%H:%M'),
                   datetime.datetime.fromisoformat(b['end']).astimezone(tz).strftime('%H:%M'))
                  for b in busy_times]

    return free_slots, busy_slots

# ------------------ 📅 CREATE EVENT ------------------
def create_event(service, title, date_str, time_str, duration_min, location=None, attendees=[]):
    tz = pytz.timezone(TIMEZONE)
    start = tz.localize(datetime.datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M"))
    end = start + datetime.timedelta(minutes=duration_min)

    event = {
        'summary': title,
        'start': {'dateTime': start.isoformat(), 'timeZone': TIMEZONE},
        'end': {'dateTime': end.isoformat(), 'timeZone': TIMEZONE},
    }
    if location:
        event['location'] = location
    if attendees:
        event['attendees'] = [{'email': a} for a in attendees]

    return service.events().insert(calendarId='primary', body=event).execute()

# ------------------ 🤖 GEMINI SETUP ------------------
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-2.5-flash")

# ------------------ 📋 PROMPT ------------------
def build_prompt(user_input, date, free_slots, busy_slots, context):
    prompt = f"""
You are a helpful smart scheduling assistant.
Today's date is {datetime.datetime.now().strftime('%Y-%m-%d')}.

Context so far:
- Title: {context.get('title')}
- Duration: {context.get('duration')}
- Preferred date: {context.get('date')}
- Time preference: {context.get('time')}
- Attendees: {context.get('attendees')}
- Location: {context.get('location')}

New user input: \"{user_input}\"
Target date: {date}
Available slots: {free_slots}
Busy slots: {busy_slots}

Instructions:
- If any required fields are missing, ask follow-up questions.
- Parse vague references like \"late next week\" or \"after my 5PM\".
- Suggest the best slot based on availability.
- Always reply in valid JSON:
{{
  "title": "",
  "date": "YYYY-MM-DD",
  "time": "HH:MM",
  "duration": 60,
  "location": "",
  "attendees": ["a@b.com"],
  "reply": "Your meeting is scheduled.",
  "reason": "Time is optimal and avoids conflict."
}}
"""
    return prompt

# ------------------ 🧠 MAIN FUNCTION ------------------
def handle_meeting():
    service = get_calendar_service()
    context = load_context() or {
        "title": None,
        "date": (datetime.datetime.now() + datetime.timedelta(days=1)).strftime('%Y-%m-%d'),
        "time": None,
        "duration": None,
        "attendees": [],
        "location": None
    }

    while True:
        user_input, lang = listen()
        if not user_input:
            speak("Sorry, I didn’t catch that. Try again.", lang)
            continue

        fallback_duration = context["duration"] if context["duration"] else 60
        free_slots, busy_slots = get_free_and_busy_slots(service, context["date"], fallback_duration)
        prompt = build_prompt(user_input, context["date"], free_slots, busy_slots, context)

        for attempt in range(2):
            try:
                response = model.generate_content(prompt)
                json_data = re.search(r'\{[\s\S]*?\}', response.text.strip())
                structured = json.loads(json_data.group())
                break
            except Exception as e:
                print("❌ Gemini Parsing Error. Retrying...", e)
                structured = {}

        # Update context
        for key in context:
            if structured.get(key):
                context[key] = structured[key]

        save_context(context)
        reply = structured.get("reply", "OK")
        reason = structured.get("reason", "")

        required = ["title", "date", "time", "duration"]
        if all(context.get(r) for r in required):
            is_free = any(s <= context["time"] < e for s, e in free_slots)
            if is_free and context['time']!= None and context['duration']!= None :
                confirm_text = f"I found a free slot at {context['time']} on {context['date']} for '{context['title']}'. Should I schedule it?"
                speak(confirm_text, lang)
                confirmation, _ = listen()
                if "yes" in confirmation.lower():
                    event = create_event(service, context["title"], context["date"], context["time"],
                                         context["duration"], context["location"], context["attendees"])
                    speak(reply + " Reason: " + reason, lang)
                    print("📅 Event Created:", event.get('htmlLink'))
                    break
                else:
                    speak("Okay, I won’t schedule it yet.", lang)
            else:
                speak(reply + " Reason: " + reason, lang)
        else:
            speak(reply, lang)



In [9]:
# ------------------ ▶️ START ------------------
if __name__ == "__main__":
    handle_meeting()


🎤 Adjusting for ambient noise...
🟢 Listening...
⏳ Transcribing...
🗣️ You said: I want to schedule a meeting for tomorrow help me
🤖 Bot: I can schedule your meeting for tomorrow, June 23rd, at 19:00. This slot works well with your preferred time and the available schedule. To finalize, could you please tell me the title of the meeting, who should be invited, and if there's a specific location?
🎤 Adjusting for ambient noise...
🟢 Listening...
⏳ Transcribing...
🗣️ You said: the title of the meeting should be energy conservation
🤖 Bot: Please tell me who should attend and where the meeting will take place. Reason: Attendees and location are required.
🎤 Adjusting for ambient noise...
🟢 Listening...
⏳ Transcribing...
❌ STT Error: 
🤖 Bot: Sorry, I didn’t catch that. Try again.
🎤 Adjusting for ambient noise...
🟢 Listening...
⏳ Transcribing...
🗣️ You said: those are not required
🤖 Bot: Your meeting is scheduled. Reason: The preferred time 19:00 for 60 minutes is available within the 18:30-00:00 

KeyboardInterrupt: 